<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

Con el siguiente algoritmo presentado se busca determinar cual fue el año en el que se procesaron la mayor cantidad de compras por parte del Estado Nacional de Argentina, estos datos son provistos de la pagina oficial del estado[1].
Para esto se utilizara el la programacion paralela por el gran caudal de datos. Se utilizara la interfaz de programación paralela OpenMP.
Con la ejecucion paralela disminuimos en gran medida los largos tiempos de procesamientos en esta clase de archivos con extensa longitud de elementos a procesar, procesaremos los diferentes archivos por separado para poder realizar comparaciones entre los resultados y asi poder determinar cual fue el año de mayor compras.

#2. Armado del ambiente

Es necesario montar Google Drive en la maquina virtual a traves de un codigo de autorizacion.
Para iniciar sesion se debe usar la siguiente cuenta de gmail:

Usuario: soa.auxiliar@gmail.com 

Contraseña: auxiliar12345

Una vez logueado se debera copiar el codigo de autorizacion que brinda google

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/

Mounted at /gdrive
/gdrive


#3. Desarrollo

In [3]:
import sys

code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include <fstream>
#include <string>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i;
  TIEMPO_INI( HTH_TOTAL )

  if( argc < 3){
      std::cerr<< " Error en los parametros: #cant_archivos pathArchivo1 ... pathArchivoN."<<std::endl;
      exit( -1 );
  }

  int cant_files = atoi(argv[1]);
  
  if(argc -2 != cant_files){
    std::cerr<< " Cantidad de archivos a procesar: "<< cant_files <<std::endl;
    std::cerr<< " Directorios pasados como parametros: "<<  argc -2 <<std::endl;
    std::cerr<< " Error en los parametros: La cantidad de archivos no concuerda con la cantidad de directorios de los archivos pasados como argumentos"<<std::endl;
    exit( -1 );
  }

  int number_of_lines[cant_files];

  int j = 0;
  for(j = 0; j < cant_files; j++){
    number_of_lines[j] = 0;
  }

//Inicio de algoritmo de forma secuencial
  std::cout <<"************INICIO algoritmo secuencial************";
  std::cout << " " << std::endl;
  TIEMPO_INI( HTH_AXPY_SEC )

  for(i = 0; i < cant_files; i++){
    number_of_lines[i] = 0;
    std::string line;
    std::ifstream myfile(argv[2 + i]);
    while (std::getline(myfile, line)){
      number_of_lines[i]++;
    }
    std::cout << "Nombre de archivo: " << argv[2 + i];
    std::cout << " " << std::endl;
    std::cout << "Lineas de texto de archivo: " << number_of_lines[i];
    std::cout << " " << std::endl;
    std::cout<<"--------------------------------------------------------"<<std::endl;
  }
  std::cout << "************FIN algoritmo secuencial************";
  std::cout << " " << std::endl;
  TIEMPO_FIN( HTH_AXPY_SEC )
//Fin de algoritmo de forma secuencial

//Inicio de algoritmo con openMP
  TIEMPO_INI( HTH_AXPY_OMP )
  std::cout << " " << std::endl;   
  std::cout <<"************INICIO algoritmo paralelo************";
  std::cout << " " << std::endl;
  #pragma omp parallel for
  for(i=0; i < cant_files; i++){
    number_of_lines[i] = 0;
    std::string line;
    std::ifstream myfile(argv[2 + i]);
    while (std::getline(myfile, line)){
      number_of_lines[i]++;
    }
    std::cout << "Nombre de archivo: " << argv[2 + i];
    std::cout << " " << std::endl;
    std::cout << "Lineas de texto de archivo: " << number_of_lines[i];
    std::cout << " " << std::endl;
    std::cout<<"--------------------------------------------------------"<<std::endl;
  }   
  std::cout <<"************FIN algoritmo paralelo************";
  std::cout << " " << std::endl;    
  TIEMPO_FIN( HTH_AXPY_OMP )
//Fin de algoritmo con openMP
  
  int max = number_of_lines[0];
  int pos = 0;
  for(i = 1; i < cant_files; i++){
    if(max < number_of_lines[i]){
      max = number_of_lines[i];
      pos = i;
    }
  }

  TIEMPO_FIN( HTH_TOTAL )
 
 std::cout<<"Archivo de mayor ventas procesadas"<<std::endl;
 std::cout<< "Nombre de archivo: " << argv[pos  + 2 ];
 std::cout<< " " << std::endl;
 std::cout<< "Maxima cantidad de lineas de texto de archivo: " << number_of_lines[pos]<<std::endl;
 std::cout<<"--------------------------------------------------------"<<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo Seuencial  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;

}
"""
try: 
  f = open('/gdrive/MyDrive/SOA_EA3/archivos.cpp', 'w')
  f.write(code)
  f.close()
except Exception:
  sys.exit("Error de I/O")


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


##3.1 Compilacion de código C Archivo

In [4]:
!g++ -o '/gdrive/MyDrive/SOA_EA3/archivos' -fopenmp '/gdrive/MyDrive/SOA_EA3/archivos.cpp'

g++: error: /gdrive/MyDrive/SOA_EA3/archivos.cpp: No such file or directory
g++: fatal error: no input files
compilation terminated.


##3.2 Ejecucion de ejecutable Archivo 

Archivos disponibles para usar como parametros:

"/gdrive/MyDrive/SOA_EA3/2014.txt"

"/gdrive/MyDrive/SOA_EA3/2015.txt"

"/gdrive/MyDrive/SOA_EA3/2016.txt"

"/gdrive/MyDrive/SOA_EA3/2017.txt"

"/gdrive/MyDrive/SOA_EA3/2018.txt"

"/gdrive/MyDrive/SOA_EA3/2019.txt"

"/gdrive/MyDrive/SOA_EA3/2020.txt"


In [5]:
%env OMP_NUM_THREADS=10
!'/gdrive/MyDrive/SOA_EA3/archivos' 7 "/gdrive/MyDrive/SOA_EA3/2014.txt" "/gdrive/MyDrive/SOA_EA3/2015.txt" "/gdrive/MyDrive/SOA_EA3/2016.txt" "/gdrive/MyDrive/SOA_EA3/2017.txt" "/gdrive/MyDrive/SOA_EA3/2018.txt" "/gdrive/MyDrive/SOA_EA3/2019.txt" "/gdrive/MyDrive/SOA_EA3/2020.txt"

env: OMP_NUM_THREADS=10
/bin/bash: /gdrive/MyDrive/SOA_EA3/archivos: No such file or directory


#4. Tabla de pasos

Tabla de ejecucion de openMP
>Lenguaje|Procesador | Funcion | Detalle
>--- | --- | --- |---
>Phyton|CPU	|code|Codigo Python que define el código C para implementar openMP
>Phyton|CPU|text_file = open()|Genera el archivo .cpp para escritura
>Phyton|CPU|text_file.write|Escribe el archivo .cpp
>Phyton|CPU|text_file.close|Cierra el archivo .cpp
>C|CPU|TIEMPO_INI( HTH_TOTAL )|Toma el tiempo inicial de la ejecucion
>C|	CPU|TIEMPO_INI( HTH_AXPY_SEC )|Toma el tiempo inicial del procesamiento secuencial
>C|	CPU|TIEMPO_FIN( HTH_AXPY_SEC )|Toma el tiempo final del procesamiento secuencial
>C|	CPU|TIEMPO_INI( HTH_AXPY_OMP )|Toma el tiempo inicial del procesamiento paralelo
>C|CPU|#pragma omp parallel for|Se define el fragemento de código que se ejecutara de forma paralela
>C|GPU|for(i=0; i < cant_files; i++)|Ejecuta de forma paralela el codigo interno del for
>C|	CPU|TIEMPO_FIN( HTH_AXPY_OMP )|Toma el tiempo final del procesamiento paralelo
>C|	CPU|TIEMPO_FIN( HTH_TOTAL )|Toma el tiempo final de la ejecucion del programa
>C|	CPU|std::cout<<""<<std::endl|Muestra de información
>bash | CPU | g++ | Compilacion de programa .cpp
>bash | CPU | !./Archivos | Ejecucion del programa

    

#5. Conclusiones

Se opto por la utilizacion de OpenMP para resolver el algoritmo ya que permite crear hilos de ejecucion de forma simple, esto es necesario ya que los archivos procesados poseen un basto nivel de informacion. Se desperdiciaría un tiempo excesivo en procesar el archivo de forma secuencial. El codigo que genera los diferentes hilos es programado en C, sin la necesidad de la libreas pthread.h. El programa se alimenta de diferentes archivos (fue necesario separlarlo en diferentes archivos ya que no permitia ser subido de forma completo) y cada uno se procesara en un hilo de ejecucion diferente. Para continuar con el ejercicio se podría realizar estadisticas sobre las empresas que vendieron mas productos al estado y por que provincias fueron realizadas.

#6. Recursos utilizados

[1].Datos.gob.ar [Archivo usado](https://datos.gob.ar/dataset/energia-refinacion-comercializacion-petroleo-gas-derivados-tablas-dinamicas/archivo/energia_6282dbf1-93d7-4fd3-a63a-61a8f8a95475)

